In [1]:
from utils import *

In [18]:
import pandas as pd

df = pd.read_csv('../data/credit_scoring.csv' , sep=';')



In [19]:
#Transformer votre jeu de données issue de pandas qui sera de type Data Frame en numpy Array  

df = df.to_numpy()


In [20]:
df

array([[   9.,    1.,   60., ...,  800.,  846.,    1.],
       [  17.,    1.,   60., ..., 1000., 1658.,    1.],
       [  10.,    0.,   36., ..., 2000., 2985.,    0.],
       ...,
       [   0.,    0.,   24., ...,  500.,  963.,    0.],
       [   0.,    1.,   48., ...,  550.,  550.,    1.],
       [   5.,    0.,   60., ..., 1350., 1650.,    1.]], shape=(4375, 14))

In [21]:
    
#séparer ensuite les variables caractéristiques de la variable à prédire (status) en deux tableaux différents. 

X = df[:, :-1]  # Toutes les colonnes sauf la dernière
y = df[:, -1]   # Seulement la dernière colonne
print(X)
print(y)

#Afficher les dimensions de vos deux tableaux X et y

print("Dimensions de X:", X.shape)
print("Dimensions de y:", y.shape)


[[   9.    1.   60. ...    0.  800.  846.]
 [  17.    1.   60. ...    0. 1000. 1658.]
 [  10.    0.   36. ...    0. 2000. 2985.]
 ...
 [   0.    0.   24. ...    0.  500.  963.]
 [   0.    1.   48. ...    0.  550.  550.]
 [   5.    0.   60. ... 1000. 1350. 1650.]]
[1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0.
 1. 1.

In [23]:

#pourcentage Analyser les propriétés de vos données : taille de l’échantillon (c.f. shape), pourcentage d’exemples positifs et négatifs.

total_samples = y.shape[0]
positive_pourcentage = np.mean(y == 1) *100
negative_pourcentage = np.mean(y == 0) *100

print(f'Taille de l\'échantillon: {total_samples}')
print(f'Pourcentage d\'exemples positifs: {positive_pourcentage:.2f}%')
print(f'Pourcentage d\'exemples négatifs: {negative_pourcentage:.2f}%')

Taille de l'échantillon: 4375
Pourcentage d'exemples positifs: 72.21%
Pourcentage d'exemples négatifs: 27.79%


In [ ]:
#Pour éviter d’avoir un résultat biaisé du classifieur que nous allons construire, séparer les données en deux parties (de taille 50% chacune) 

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)
print("Dimensions de X_train:", X_train.shape)
print("Dimensions de y_train:", y_train.shape)
print("Dimensions de X_test:", X_test.shape)
print("Dimensions de y_test:", y_test.shape)


Dimensions de X_train: (2187, 13)
Dimensions de y_train: (2187,)
Dimensions de X_test: (2188, 13)
Dimensions de y_test: (2188,)


In [26]:
total_samples = y_train.shape[0]
positive_pourcentage = np.mean(y_train== 1) *100
negative_pourcentage = np.mean(y_train == 0) *100

print(f'Taille de l\'échantillon: {total_samples}')
print(f'Pourcentage d\'exemples positifs: {positive_pourcentage:.2f}%')
print(f'Pourcentage d\'exemples négatifs: {negative_pourcentage:.2f}%')

Taille de l'échantillon: 2187
Pourcentage d'exemples positifs: 72.02%
Pourcentage d'exemples négatifs: 27.98%
